In [ ]:
# https://school.programmers.co.kr/learn/courses/30/lessons/60063
# https://www.youtube.com/watch?v=XLa2JL_UCko 참고한 해설 유튜브

In [ ]:
# 해설 유튜브의 내용을 그대로 정리한 것입니다.

import queue

class Point : 
    def __init__(self, row, col, dir, time):
        self.row = row
        self.col = col
        self.dir = dir # 방향
        self.time = time # 현재걸린시간

# 회전을 위해 상->하->좌->우 1씩 증가하는 값으로 지정
UP,RIGHT,DOWN,LEFT = 0 , 1 , 2, 3
D = ((-1,0),(1,0),(0,-1),(0,1)) # 상하좌우 이동
# 로테이션을 위한 D / 시계0(상하좌우) , 반시계1(상하좌우) / 행열의 값 변화
Drot = (((1,1),(1,-1),(-1,-1),(-1,1)),((1,-1),(-1,-1),(-1,1),(1,1)))
# 회전이 가능한지 체크 / 중심축의 대각선들이 0인지 확인
Dcor = (((-1,1),(1,1),(1,-1),(-1,-1)),((-1,-1),(-1,1),(1,1),(1,-1)))

Board = []
N = 0
Q = queue.Queue()
Visited = [[[False for _ in range(4)] for _ in range(100)] for _ in range(100)]

def isValid(block):
    for pt in block:
        # 보드를 벗어난경우 / 
        if pt.row < 0 or pt.row > N-1 or pt.col < 0 or pt.col > N-1:
            return False
        # 벽을 만나는 경우
        if Board[pt.row][pt.col] == 1:
            return False
        # 이미 방문한 곳인 경우
        if Visited[pt.row][pt.col][pt.dir]:
            return False
    # 위의 경우에 모두 해당하지 않는다면
    return True

def rotate(curr, ccw, idx): #현재위치, 회전방향, 회전축
    newPt=[]
    # 회전축이 왼쪽0이면 0,1 / 회전축이 오른쪽1이면 1,0
    a = idx
    b = (idx + 1) % 2 
    dir =  curr[a].dir
    
    # 첫번째 좌표(중심축) 중심축의 위치는 바뀌지않음 
    # 시계방향 회전 / 상하좌우 0123 이므로 4로 나누는 연산
    # 반시계 회전 / 1을 빼는 연산 대신 3을 더하는 연산. 음수가되면 나머지연산이 불가능 (3 if ccw else 1)
    newPt.append(Point(curr[a].row, curr[a].col, \
                        (curr[a].dir + (3 if ccw else 1)) % 4, curr[a].time +1 ))
    # 두번째 좌표
    newPt.append(Point(curr[b].row + Drot[ccw][dir][0], curr[b].col + Drot[ccw][dir][1], \
                        (curr[b].dir + ( 3 if ccw else 1)) % 4, curr[b].time + 1 ))
    
    # 자리가 바뀌었으니 재검증
    if isValid(newPt) == False :
        return 0
    # 모서리에 벽이있나 검증
    if Board[curr[a].row + Dcor[ccw][dir][0]][curr[a].col + Dcor[ccw][dir][1]] == 1 : 
        return 0 
    
    # 방문여부 마킹
    for pt in newPt :
        # 목적지에 이미 도착했을 경우
        if pt.row == N-1 and pt.col == N-1 :
            return pt.time
        Visited[pt.row][pt.col][pt.dir] = True
        
    Q.put(newPt)
    
    return 0

    
def solution(board):
    global Board, N, Visited, Q
    Board = board
    N = len(board)
    
    Q.put([Point(0,0,RIGHT,0),Point(0,1,LEFT,0)])
    Visited[0][0][RIGHT] = True
    Visited[0][1][LEFT] = True
    
    while Q:
        curr = Q.get()
        
        for j in range(4):
            newPt = []
            # 상하좌우 이동
            for i in range(2):
                newPt.append(Point(curr[i].row + D[j][0], curr[i].col + D[j][1], \
                    curr[i].dir, curr[i].time +1 ))
            
            # 벽(1)이나 벗어나는지 확인
            if isValid(newPt) == False:
                continue
            
            # 방문 여부 마킹
            for pt in newPt:
                # 목적지에 도착한 경우
                if pt.row == N-1 and pt.col == N-1:
                    return pt.time
                
                Visited[pt.row][pt.col][pt.dir] = True

            Q.put(newPt)
        
        # 회전 -> 4가지 경우 존재
        for ccw in range(2): # 시계0 반시계1
            for i in range(2): # 회전축이 첫번째인지 두번째인지
                ret = rotate(curr,ccw, i) # 현재위치, 회전방향, 회전축
                if ret:
                    return ret;
    # 검증과정에서 중간중간 도착했는지 확인하게 되므로 최종 리턴값은 큰 의미가 없음
    return 0